In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#Metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
%matplotlib inline
#Clustering
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
#Otros
from sklearn.multioutput import MultiOutputRegressor

### CSV Loading

In [2]:
dataset = pd.read_csv('./data/t_data/anime_cleaned_treated_v2_1.csv')

In [3]:
dataset.head()

,Genres,Source,Type,Episodes,Decimal,Romance,Shoujo,Ecchi,Shounen,Fantasy,...,Mistery,Thriller,Supernatural,Gore,SciFi,Action,Otro,Popularity,Score,Favorites
0,"Comedy, Romance, Shounen, Supernatural",6,5,12,147,1,0,0,1,0,...,0,0,1,0,0,0,0,231,7.63,2809
1,"Comedy, Romance, Shounen",6,5,26,19,1,0,0,1,0,...,0,0,0,0,0,0,0,366,7.89,2579
2,"Comedy, Romance, Fantasy, Shoujo",6,5,51,519,1,1,0,0,1,...,0,0,0,0,0,0,0,1173,7.55,802
3,"Comedy, Romance, Fantasy",9,5,38,7,1,0,0,0,1,...,0,0,0,0,0,0,0,916,8.21,3344
4,"Comedy, Romance, Shounen",6,5,25,19,1,0,0,1,0,...,0,0,0,0,0,0,0,426,8.67,2082


### Training

In [4]:
data = dataset.values
X = data[:int(len(data)*0.7),1:22]
y = data[:int(len(data)*0.7),22:24]
X_t = data[int(len(data)*0.7):,1:22]
y_t = data[int(len(data)*0.7):,22:24]
print y[0]

[231 7.63]


In [5]:
DT_R = DecisionTreeRegressor(max_depth=5)
RF_R = RandomForestRegressor(n_estimators=40)
GB_R = MultiOutputRegressor(GradientBoostingRegressor())
#np.array([6,5,12,147]).reshape(1,-1)

In [6]:
DT_R.fit(X,y)
RF_R.fit(X,y)
GB_R.fit(X,y)

MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
           n_jobs=1)

In [7]:
for i in range(10):
    print "============="
    print RF_R.predict(X_t[i].reshape(1,-1))
    print DT_R.predict(X_t[i].reshape(1,-1))
    print GB_R.predict(X_t[i].reshape(1,-1))
    print y_t[i]

[[ 1721.49166667     7.20175   ]]
[[ 3231.96540362     6.91583196]]
[[ 2334.3658176      7.44853863]]
[40 8.56]
[[ 4801.65125       6.6507875]]
[[ 5715.97260274     6.49856164]]
[[ 5382.30646749     6.62069068]]
[3370 7.23]
[[ 3920.15        6.95375]]
[[ 3325.4379562      7.04386861]]
[[ 3713.54253324     7.08721126]]
[5068 6.01]
[[ 2527.23548535     6.70214591]]
[[ 2011.57080132     7.22140505]]
[[ 1784.57310886     6.37370076]]
[2088 5.57]
[[  1.00264571e+04   6.22655417e+00]]
[[  7.20313433e+03   6.75044776e+00]]
[[  7.80029690e+03   6.71543603e+00]]
[10312 7.17]
[[  7.11880524e+03   6.85164524e+00]]
[[  7.20313433e+03   6.75044776e+00]]
[[ 6250.39246821     7.0150596 ]]
[9390 5.92]
[[ 2417.1       7.912]]
[[ 3913.33333333     7.36469697]]
[[ 3959.41475762     7.58482128]]
[1621 8.51]
[[ 1237.3         6.33725]]
[[ 2011.57080132     7.22140505]]
[[ 617.57260891    6.90118585]]
[540 5.71]
[[ 446.79027778    7.84235069]]
[[ 3231.96540362     6.91583196]]
[[ 1387.48239655     7.5988316

In [18]:
s = cross_val_score(DT_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 2205.406 (+/- 694.87338)


In [9]:
s = cross_val_score(RF_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 2056.187 (+/- 643.77728)


In [10]:
s = cross_val_score(GB_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 1980.423 (+/- 790.63669)


### Arbitrary new anime prediction test

In [19]:
new_anime = np.array([6,5,24,50822,1,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0]).reshape(1,-1)

In [20]:
GB_R.predict(new_anime)

array([[ 2545.09202523,     6.90995925]])

### Object persistence

In [21]:
import pickle
with open('regressors.pkl', 'wb') as output:
    pickle.dump(DT_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(RF_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(GB_R, output, pickle.HIGHEST_PROTOCOL)
